# The developer storage API

The typical user will never need to use this API.

The only functionality provided is writing and reading objects and files.

## Local mode

We'll write files to a directory "data" in our home:

In [ ]:
!lndb login testuser1 --password "cEvcwMJFX4OwbsYVaMt2Os6GxxGgDUlBGILs2RyS"
!lndb init --storage mydatatmp

In [ ]:
from nbproject import header
from lndb_setup import init
import lamindb.dev as lndv
from lamindb import settings
import scanpy as sc
import pytest  # only needed to demonstrate exceptions

header()

Let's set up a test data object and a file.

In [ ]:
adata = sc.datasets.pbmc68k_reduced()

In [ ]:
lndv.object.anndata_to_h5ad(adata, "test-file.h5ad")

As `h5ad` files have a 1:1 mapping with `AnnData` objects, we can type-check the user input.

If we try to write a DataFrame to an `.h5ad`, this will raise a `TypeError`.

In [ ]:
with pytest.raises(TypeError):
    lndv.object.anndata_to_h5ad(adata.to_df())

In [ ]:
(settings.instance.storage_root / "test-file.h5ad").unlink()
settings.instance._sqlite_file.unlink()

## Cloud mode on S3

Let us now configure storage in the cloud, here, on AWS S3.

We need a cache directory for backing data from the cloud locally and provide it at setup in addition to the S3 bucket.

(In the future, we'll make this optional and use a system cache directory.)

If we weren't in a single notebook, here, you could also call `lndb init --storage "s3://lamin0"` instead of the corresponding python function below. However, the CLI starts a new python process with a new settings class, which doesn't update our current in-memory settings class here.

In [ ]:
init(storage="s3://lamin0")

Otherwise, the API works as before.

In [ ]:
settings.instance

In [ ]:
from lndb_setup import settings

In [ ]:
settings.instance.cloud_storage

In [ ]:
assert settings.instance.cloud_storage == True

In [ ]:
cache_file = lndv.object.anndata_to_h5ad(adata, "test-file.h5ad")

In [ ]:
cache_file

As does reading.

In [ ]:
adata = lndv.file.h5ad_to_anndata("test-file.h5ad")

If the cache file disappears,

In [ ]:
list(cache_file.parent.glob("*"))

In [ ]:
cache_file.unlink()

In [ ]:
list(cache_file.parent.glob("*"))

The data in the cloud will be auto-downloaded:

In [ ]:
adata = lndv.file.h5ad_to_anndata("test-file.h5ad")  # Just temporarily

In [ ]:
list(cache_file.parent.glob("*"))